In [ ]:
import sys
print(sys.version)
%pwd
#git clone https://github.com/fastai/fastai.git
#cd fastai
#conda env update
#conda activate fastai

In [ ]:
#!pip install fastai
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.structured import *
from fastai.column_data import *

In [ ]:
Path= %pwd

In [ ]:
PATH = "data/"

In [ ]:
pd.read_csv(PATH + 'sample_submission.csv').head()

In [ ]:
from IPython.display import HTML, display
table_names = ['test', 'train']
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]

for t in tables: display(t.head())

In [ ]:
test, train = tables

In [ ]:
len(train),len(test)

In [ ]:
add_datepart(train, "date", drop=False)

In [ ]:
add_datepart(test, "date", drop=False)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
cat_vars = ['store', 'item', 'Dayofweek', 'Dayofyear', 'Year', 'Month', 'Day', 'Week', 
            'Is_month_end',	'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']

contin_vars = []

n = len(train); n

In [ ]:
dep = 'sales'
train_df = train[cat_vars+contin_vars+[dep, 'date']].copy()

In [ ]:
test[dep] = 0
test_df = test[cat_vars+contin_vars+[dep, 'date', 'id']].copy()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
for v in cat_vars: train_df[v] = train_df[v].astype('category').cat.as_ordered()

In [ ]:
for v in cat_vars: test_df[v] = test_df[v].astype('category').cat.as_ordered()

In [ ]:
for v in contin_vars:
    train_df[v] = train_df[v].fillna(0).astype('float32')
    test_df[v] = test_df[v].fillna(0).astype('float32')

In [ ]:
train_df.info()

In [ ]:
train_df['date'].head()

In [ ]:
train_df.isnull().any()

In [ ]:
samp_size = n
train_index = train_df.set_index("date")

In [ ]:
#idxs = get_cv_idxs(n, val_pct=150000/n)
#train_index = train_df.iloc[idxs].set_index("date")
#samp_size = len(train_index); 

In [ ]:
samp_size

In [ ]:
df, y, nas = proc_df(train_index, 'sales')

In [ ]:
lowest = y.argmin()
while y[lowest] <= 0:
    print(str(lowest) + ':' + str(y[lowest]))
    y[lowest] = 1
    lowest = y.argmin()

In [ ]:
yl = np.log(y)

In [ ]:
df.info()

In [ ]:
test_index = test_df.set_index("date")

In [ ]:
df_test, _, nas = proc_df(test_index, 'sales', skip_flds=['id'], na_dict=nas)

In [ ]:
df_test.info()

In [ ]:
df.head()

In [ ]:
train_df['date'].head()

In [ ]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [ ]:
val_idx = np.flatnonzero((df.index<=datetime.datetime(2017,12,31)) & (df.index>=datetime.datetime(2017,10,1)))

In [ ]:
len(df)

In [ ]:
len(val_idx)

In [ ]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = torch.from_numpy(targ).cuda()
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

def exp_smape(y_pred, y_true):
    y_pred = inv_y(y_pred)
    y_true = inv_y(y_true)
    print(y_pred)
    print(y_true)
    #print(torch.mean(np.abs((y_true - y_pred) / y_true)) * 100)
    return torch.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [ ]:
len(val_idx), len(yl)

In [ ]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)

In [ ]:
cat_sz = [(c, len(train_index[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot()

In [ ]:
lr = 1e-4 #0.0005

In [ ]:
m.fit(lr, 3, metrics=[exp_smape])

In [ ]:
m.save("forecast_3E")

In [ ]:
m.load("forecast_3E")

In [ ]:
m.fit(lr, 5, metrics=[exp_smape])

In [ ]:
m.save("forecast_8E")

In [ ]:
m.load("forecast_8E")

In [ ]:
m.fit(lr, 2, metrics=[exp_smape], cycle_len = 2, cycle_mult = 2)

In [ ]:
m.save("forecast_8E_6_ER")

# Change to whole dataset

In [ ]:
#current contender: m.save("forecast_full_4E_9ER")

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)

In [ ]:
m.load("forecast_8E_6_ER")

In [ ]:
m.fit(lr, 1, metrics=[exp_smape])

In [ ]:
m.fit(lr, 1, metrics=[exp_smape], cycle_len = 3)

In [ ]:
m.save("forecast_full_4E_2")

In [ ]:
m.load("forecast_full_4E_2_9ER")

In [ ]:
m.fit(lr, 2, metrics=[exp_smape], cycle_len = 4, cycle_mult = 1)

In [ ]:
m.save("forecast_full_4E_6ER")

In [ ]:
m.load("forecast_full_4E_6ER")

# Test Set

In [ ]:
x,y=m.predict_with_targs()
x = torch.from_numpy(x)
y = torch.from_numpy(y)
exp_smape(x,y)

#pred_test=m.predict(True)

In [ ]:
log_preds = m.predict(True)

In [ ]:
log_preds

In [ ]:
log_test = np.exp(log_preds)

In [ ]:
log_test

In [ ]:
test.head()

In [ ]:
test['sales'] = log_test.round().astype(int)

In [ ]:
test.head()

In [ ]:
test = test.drop('date', 1)
test = test.drop('store', 1)
test = test.drop('item', 1)

In [ ]:
test.head()

In [ ]:
csv_fn=f'{PATH}submission.csv'

test.to_csv(csv_fn, index=False)

In [ ]:
!chmod 600 C:/Users/chrun/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c demand-forecasting-kernels-only -f data/submission.csv -m "First Try"